In [1]:
import os
import shutil
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt

In [5]:
df = pd.read_csv('20240527-higher table.csv').sort_values('name', ignore_index=True)
bg_df = pd.read_csv('20240528-background table.csv').sort_values('name', ignore_index=True)
fake_df = pd.read_csv('20240528-fake table.csv').sort_values('name', ignore_index=True)
ri_df = pd.read_csv('20240528-RI table.csv').sort_values('name', ignore_index=True)
ro_df = pd.read_csv('20240528-RO table.csv').sort_values('name', ignore_index=True)

df = bg_df[['name']]
df['class'] = None
df.loc[df['name'].str.contains('RO'), 'class'] = 'RO'
df['class'] = df['class'].fillna('RI')

# best score in private
df['rule1'] = 'Tony1'
rule = fake_df['Henry1'].le(fake_df['Tony1'])
sub_rule_1 = df['class'].eq('RI') & ri_df['Henry1'].ge(ri_df['Tony1'])
sub_rule_2 = df['class'].eq('RO') & ro_df['Henry1'].ge(ro_df['Tony1'])
rule &= sub_rule_1 | sub_rule_2
df.loc[rule, 'rule1'] = 'Henry1'

df['rule2'] = 'Tony1'
sub_rule_1 = fake_df['Tony2'].le(fake_df['Tony1'])
sub_rule_1 |= df['class'].eq('RI') & ri_df['Tony2'].ge(ri_df['Tony1'])
sub_rule_1 |= df['class'].eq('RO') & ro_df['Tony2'].ge(ro_df['Tony1'])
sub_rule_2 = fake_df['Henry1'].le(fake_df['Tony1']) & fake_df['Henry1'].le(fake_df['Tony2'])
sub_rule_2_ri = df['class'].eq('RI') & ri_df['Henry1'].ge(ri_df['Tony1']) & ri_df['Henry1'].ge(ri_df['Tony2'])
sub_rule_2_ro = df['class'].eq('RO') & ro_df['Henry1'].ge(ro_df['Tony1']) & ro_df['Henry1'].ge(ro_df['Tony2'])
sub_rule_2 &= sub_rule_2_ri | sub_rule_2_ro
df.loc[sub_rule_1, 'rule2'] = 'Tony2'
df.loc[sub_rule_2, 'rule2'] = 'Henry1'

df['rule3'] = 'Tony1'
sub_rule_1 = fake_df['Tony2'].le(fake_df['Tony1'])
sub_rule_1 |= df['class'].eq('RI') & ri_df['Tony2'].ge(ri_df['Tony1'])
sub_rule_1 |= df['class'].eq('RO') & ro_df['Tony2'].ge(ro_df['Tony1'])
df.loc[sub_rule_1, 'rule3'] = 'Tony2'
sub_rule_2 = fake_df['Henry1'].le(df.apply(lambda x: fake_df.loc[x.name, x['rule3']], axis=1))
sub_rule_2_ri = df['class'].eq('RI') & ri_df['Henry1'].ge(df.apply(lambda x: ri_df.loc[x.name, x['rule3']], axis=1))
sub_rule_2_ro = df['class'].eq('RO') & ro_df['Henry1'].ge(df.apply(lambda x: ro_df.loc[x.name, x['rule3']], axis=1))
sub_rule_2 &= sub_rule_2_ri | sub_rule_2_ro
df.loc[sub_rule_2, 'rule3'] = 'Henry1'

df['rule4'] = 'Henry1'
rule1 = fake_df['Henry2'].le(fake_df['Henry1'])
rule2 = df['class'].eq('RI') & ri_df['Henry2'].ge(ri_df['Henry1'])
rule3 = df['class'].eq('RO') & ro_df['Henry2'].ge(ro_df['Henry1'])
rule = rule1 & (rule2 | rule3)
df.loc[rule, 'rule4'] = 'Henry2'
rule = fake_df['Tony1'].le(df.apply(lambda x: fake_df.loc[x.name, x['rule4']], axis=1))
rule |= df['class'].eq('RI') & ri_df['Tony1'].ge(df.apply(lambda x: ri_df.loc[x.name, x['rule4']], axis=1))
rule |= df['class'].eq('RO') & ro_df['Tony1'].ge(df.apply(lambda x: ro_df.loc[x.name, x['rule4']], axis=1))
df.loc[rule, 'rule4'] = 'Tony1'

df.groupby('rule1').count()

,name,fake,background,RI,RO,class,rule2,rule3,rule4
rule1,,,,,,,,,
Henry1,192,192,192,192,192,192,192,192,192
Tony1,1248,1248,1248,1248,1248,1248,1248,1248,1248


In [6]:
folder = {
    'Henry1': '/app/DP_GAN/private/UAV_instance_512_z128_400/best/image',
    'Henry2': '/app/DP_GAN/private2/UAV_instance_512_z128_400/best/image',
    'Tony1': '/app/DP_GAN/20240521ImproveMaskSampleC1.4SampleStep1000',
    'Tony2': '/app/DP_GAN/20240521ImproveMaskSampleC1SampleStep1000',
}
out_folder = 'ensemble4'
# df.apply(lambda x: shutil.copyfile(f"{folder[x['rule1']]}/{x['name']}.jpg", f"{out_folder}/{x['name']}.jpg"), axis=1)